# Working with European whitefish data

In this notebook I am going to work with European whitefish data from my phd project. The aims of this mini-project are:

* download reference genome from ncbi
* import vcfs using hail
* run pca and other analysis

All work will be conducted using python libraries (and cmd tools if necessary).

In [ ]:
# pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 8.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for biopython: filename=biopython-1.79-cp310-cp310-macosx_10_9_x86_64.whl size=2262217 sha256=14e94313d0d9065b8059274c7811f1df751b7679e2e1755ad7f3c3fd33eeb7b6
  Stored in directory: /Users/marcocrotti/Library/Caches/pip/wheels/7c/46/dd/3e2f23acbbe0dd5b3744d069ad905e0218a29c879291beeef5
Successfully built biopython
Note: you may need to restart the kernel to use updated packages.


## Download reference genome

To download the reference genome, I am using biopython and Entrez.

In [5]:
# import Entrez functionalities to download data from ncbi

from Bio import Entrez
from Bio import  SeqIO

Entrez.email = "marco.crotti9@gmail.com"


In [51]:
# Search for assembly
handle = Entrez.esearch(db="assembly", term="Coregonus sp. 'balchen'")
record = Entrez.read(handle)
record["IdList"]

['6278041', '4234101']

Apparently, `entrez.efetch` cannot access the 'assembly' database. Instead I am searching each chromosome in the 'nucleotide' database.

In [130]:
handle = Entrez.esearch(db="nucleotide", term="Coregonus sp. 'balchen' genome assembly ", retmax = 100, idtype='acc')
record = Entrez.read(handle)
record["IdList"].reverse()

In [161]:
for ID in record["IdList"][:10]:
    print(ID)

LR778253.1
LR778254.1
LR778255.1
LR778256.1
LR778257.1
LR778258.1
LR778259.1
LR778260.1
LR778261.1
LR778262.1


Here I am creating an empty array, looping through the results of `efetch`, append each record to the array, and finally print the concatenated fasta file (i.e. the genome).

In [ ]:
handle = Entrez.efetch(db="nucleotide", rettype="fasta", retmode="text", id=record["IdList"], retmax = 100, idtype='acc')
records = []
for seq_record in SeqIO.parse(handle, "fasta"):
    print(seq_record.id)
    records.append(seq_record)
  # SeqIO.write(seq_record, seq_record.id+".fa", "fasta")  this lines will save each individual chromosome
SeqIO.write(records, "genome.fa", "fasta")

LR778253.1
LR778254.1
LR778255.1
LR778256.1
LR778257.1
LR778258.1
LR778259.1
LR778260.1
LR778261.1
LR778262.1
LR778263.1
LR778264.1
LR778265.1
LR778266.1
LR778267.1
LR778268.1
LR778269.1
LR778270.1
LR778271.1
LR778272.1
LR778273.1
LR778274.1
LR778275.1
LR778276.1
LR778277.1
LR778278.1
LR778279.1
LR778280.1
LR778281.1
LR778282.1
LR778283.1
LR778284.1
LR778285.1
LR778286.1
LR778287.1
LR778288.1
LR778289.1
LR778290.1
LR778291.1
LR778292.1


40

### Index reference genome

I am using `pyfaidx` to index the reference genome

In [164]:
# pip install pyfaidx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyfaidx: filename=pyfaidx-0.7.1-py3-none-any.whl size=27730 sha256=45b576dd700aff9eb1fc73ee76e23c829ad9deb54103e41b6e2abac119e1b50b
  Stored in directory: /Users/marcocrotti/Library/Caches/pip/wheels/bf/25/63/cec5986dc9fee49682e532e5fb1faab7b6aab6fb3d2b9ede4c
Successfully built pyfaidx
Note: you may need to restart the kernel to use updated packages.


In [165]:
from pyfaidx import Faidx
fa = Faidx('genome.fa')

In [167]:
fa.index

OrderedDict([('LR778253.1',
              IndexRecord(rlen=93459789, offset=67, lenc=60, lenb=61, bend=95017520, prev_bend=0)),
             ('LR778254.1',
              IndexRecord(rlen=43329510, offset=95017587, lenc=60, lenb=61, bend=139069256, prev_bend=95017520)),
             ('LR778255.1',
              IndexRecord(rlen=42764345, offset=139069323, lenc=60, lenb=61, bend=182546408, prev_bend=139069256)),
             ('LR778256.1',
              IndexRecord(rlen=92224161, offset=182546475, lenc=60, lenb=61, bend=276307706, prev_bend=182546408)),
             ('LR778257.1',
              IndexRecord(rlen=45591172, offset=276307773, lenc=60, lenb=61, bend=322658798, prev_bend=276307706)),
             ('LR778258.1',
              IndexRecord(rlen=43692974, offset=322658865, lenc=60, lenb=61, bend=367080056, prev_bend=322658798)),
             ('LR778259.1',
              IndexRecord(rlen=65391737, offset=367080123, lenc=60, lenb=61, bend=433561723, prev_bend=367080056)),
          

## Import vcfs with Hail

### Import reference genome

In [1]:
import hail as hl

from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

In [2]:
hl.init()

2022-08-19 15:04:37 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-08-19 15:04:37 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.2, running with Spark 3.1.3.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.1.101:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.95-513139587f57
LOGGING: writing to /Users/marcocrotti/Desktop/work/hail/whitefish_project/hail-20220819-1504-0.2.95-513139587f57.log


In [3]:
whitefish_genome = hl.ReferenceGenome.from_fasta_file("whitefish",
                                                     fasta_file = "genome.fa",
                                                     index_file = "genome.fa.fai")

In [4]:
whitefish_genome.lengths

{'LR778253.1': 93459789,
 'LR778254.1': 43329510,
 'LR778255.1': 42764345,
 'LR778256.1': 92224161,
 'LR778257.1': 45591172,
 'LR778258.1': 43692974,
 'LR778259.1': 65391737,
 'LR778260.1': 68138733,
 'LR778261.1': 65193448,
 'LR778262.1': 60468309,
 'LR778263.1': 63177489,
 'LR778264.1': 63881516,
 'LR778265.1': 57740044,
 'LR778266.1': 47256133,
 'LR778267.1': 55641933,
 'LR778268.1': 54025139,
 'LR778269.1': 54216998,
 'LR778270.1': 51949489,
 'LR778271.1': 59907985,
 'LR778272.1': 54335267,
 'LR778273.1': 52945597,
 'LR778274.1': 56862223,
 'LR778275.1': 52020451,
 'LR778276.1': 50329371,
 'LR778277.1': 51033154,
 'LR778278.1': 50922480,
 'LR778279.1': 48683376,
 'LR778280.1': 46671285,
 'LR778281.1': 48977775,
 'LR778282.1': 48675208,
 'LR778283.1': 48446552,
 'LR778284.1': 44616205,
 'LR778285.1': 44662967,
 'LR778286.1': 40727438,
 'LR778287.1': 42609905,
 'LR778288.1': 42009912,
 'LR778289.1': 43663377,
 'LR778290.1': 36774138,
 'LR778291.1': 33962415,
 'LR778292.1': 1094305}

### Import vcfs

### Manipulate vcf files in wrong format

In [117]:
! bgzip -d *.vcf.gz

In [55]:
import re
import os

def clean_vcf(filename):
    
    basename = os.path.splitext(os.path.basename(filename))[0]
    with open(filename,'r') as file:
        with open(basename+".fixed.vcf", "wt") as fout:
            f=file.read()
            f_chr = re.sub(".+[|]","", f)
            fout.write(re.sub("\"", "", f_chr))

In [56]:
clean_vcf("lom.recode.vcf")
clean_vcf("eck.recode.vcf")

In [58]:
! bgzip *.fixed.vcf

In [121]:
! bcftools index lom.recode.fixed.vcf.gz 
! bcftools index eck.recode.fixed.vcf.gz 

### Convert from vcf to mt format

In [5]:
hl.import_vcf("eck.recode.fixed.vcf.gz",
              force_bgz=True,
              reference_genome="whitefish").write("eck.mt", overwrite=True)

eck_mt = hl.read_matrix_table("eck.mt")

2022-08-19 15:05:43 Hail: INFO: Coerced sorted dataset              (0 + 1) / 1]
2022-08-19 15:05:47 Hail: INFO: wrote matrix table with 3712 rows and 77 columns in 1 partition to eck.mt


In [6]:
hl.import_vcf("lom.recode.fixed.vcf.gz",
              force_bgz=True,
              reference_genome="whitefish").write("lom.mt", overwrite=True)

lom_mt = hl.read_matrix_table("lom.mt")

2022-08-19 15:05:50 Hail: INFO: Coerced sorted dataset
2022-08-19 15:05:53 Hail: INFO: wrote matrix table with 6333 rows and 110 columns in 1 partition to lom.mt


### Merge vcf

In [ ]:
merged_mt = eck_mt.union_cols(lom_mt, row_join_type='outer')
merged_mt.count()

(8184, 187)

In [ ]:
merged_mt.describe()

### QC and filter

#### Filter by variant

In [8]:
merged_mt = hl.variant_qc(merged_mt)

In [9]:
merged_mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<whitefish>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AD: array<int32>, 
        AF: array<float64>, 
        DP: int32, 
        NS: int32, 
        loc_strand: str
    }
    'variant_qc': struct {
        dp_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        gq_stats: struct {
            mean: float64, 
            stdev: float64, 
            min: float64, 
            max: float64
        }, 
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        homozygote_count: array<int32>, 
        call_rate: float64, 
        n_called: int64, 
        n_not_called: int64, 
        n_filtered: int64

In [10]:
# Plot call rate

p = hl.plot.histogram(merged_mt.variant_qc.call_rate, range = (.50, 1), legend = 'Variant Call Rate')
show(p)

In [11]:
# Plot mean depth

p = hl.plot.histogram(merged_mt.variant_qc.dp_stats.mean, legend = 'Variant Mean Depth')
show(p)

In [12]:
# Plot mean GQ

p = hl.plot.histogram(merged_mt.variant_qc.gq_stats.mean, legend = 'Variant Mean GQ')
show(p)

In [13]:
# Filter variants with call rate < 60%

merged_filtered_mt = merged_mt.filter_rows((merged_mt.variant_qc.call_rate > 0.6) & (merged_mt.variant_qc.dp_stats.mean < 40))
print('After filter, %d/8184 variants remain.' % merged_filtered_mt.count_rows())

After filter, 1826/8184 variants remain.


#### Filter by sample

In [14]:
merged_filtered_mt = hl.sample_qc(merged_filtered_mt)

In [15]:
# Plotting call rate

p = hl.plot.histogram(merged_filtered_mt.sample_qc.call_rate, range = (.60,1), legend = 'Call Rate')
show(p)

In [16]:
# Filter samples with call rate < 75%

merged_filtered_mt = merged_filtered_mt.filter_cols(merged_filtered_mt.sample_qc.call_rate >= 0.75)
print('After filter, %d/187 samples remain' % merged_filtered_mt.count_cols())

After filter, 180/187 samples remain


## Exploratory analyses

In [17]:
# read in popmap

popmap = hl.import_table("popmap.txt", impute=True, key='id')
popmap.show()

2022-08-19 15:07:20 Hail: INFO: Reading table to impute column types
2022-08-19 15:07:21 Hail: INFO: Finished type imputation
  Loading field 'id' as type str (imputed)
  Loading field 'lake' as type str (imputed)
  Loading field 'system' as type str (imputed)


,,
id,lake,system
str,str,str
"""ELT08300""","""Allt na Lairige""","""Lomond"""
"""ELT08301""","""Allt na Lairige""","""Lomond"""
"""ELT08302""","""Allt na Lairige""","""Lomond"""
"""ELT08303""","""Allt na Lairige""","""Lomond"""
"""ELT08304""","""Allt na Lairige""","""Lomond"""
"""ELT08305""","""Allt na Lairige""","""Lomond"""
"""ELT08306""","""Allt na Lairige""","""Lomond"""
"""ELT08308""","""Allt na Lairige""","""Lomond"""


In [18]:
# combine genotypic and phenotypic data

merged_filtered_mt = merged_filtered_mt.annotate_cols(pheno = popmap[merged_filtered_mt.s])


In [19]:
# PCA

eigenvalues, pcs, _ = hl.hwe_normalized_pca(merged_filtered_mt.GT)
pcs.show(5)

2022-08-19 15:07:38 Hail: INFO: hwe_normalize: found 1826 variants after filtering out monomorphic sites.
2022-08-19 15:07:42 Hail: INFO: pca: running PCA with 10 components... + 1) / 2]


,
s,scores
str,array<float64>
"""ELT08300""","[-2.19e-01,-9.17e-02,-1.44e-01,-1.69e-01,5.43e-01,6.25e-03,1.37e-01,-2.62e-02,9.95e-02,2.77e-01]"
"""ELT08301""","[-2.20e-01,-1.59e-01,-1.20e-01,2.00e-01,-2.32e-01,1.27e-02,-8.80e-02,-1.65e-01,-6.66e-02,-2.97e-02]"
"""ELT08302""","[-1.98e-01,-1.62e-01,-1.22e-01,2.78e-01,-2.89e-01,-2.86e-02,-2.69e-02,-2.51e-01,5.99e-02,-1.06e-01]"
"""ELT08303""","[-1.81e-01,-7.59e-02,7.36e-02,-1.10e-03,1.91e-01,-1.24e-01,-8.01e-02,1.35e-01,-8.04e-03,-3.67e-01]"
"""ELT08304""","[-2.09e-01,-1.23e-01,-6.97e-02,-1.09e-01,-1.58e-02,-4.70e-02,1.14e-01,-1.89e-02,1.50e-01,-8.77e-02]"


In [20]:
merged_filtered_mt = merged_filtered_mt.annotate_cols(scores = pcs[merged_filtered_mt.s].scores)

In [21]:
# Set color mapper for plot
from bokeh.models import CategoricalColorMapper

lakes = ['Allt na Lairige','Shira','Lomond','Sloy','Carron','Eck','Glashan','Tarsan']
palette = ['#a4dede','#287c71','gray','#6c966f','#008396','gray','#9b2915','#fe7c73']

mapper = CategoricalColorMapper(palette=palette, 
                                factors=lakes)


In [23]:
# plot PC1 and PC2

p = hl.plot.scatter(merged_filtered_mt.scores[0],
                    merged_filtered_mt.scores[1],
                    label = {'Lake':merged_filtered_mt.pheno.lake},
                    colors= {'Lake': mapper},
                    title = 'PCA', xlabel = 'PC1', ylabel = 'PC2',
                    size = 8)
show(p)

In [24]:
# plot PC1 and PC3

p = hl.plot.scatter(merged_filtered_mt.scores[0],
                    merged_filtered_mt.scores[2],
                    label = {'Lake':merged_filtered_mt.pheno.lake},
                    colors= {'Lake': mapper},
                    title = 'PCA', xlabel = 'PC1', ylabel = 'PC3',
                    size = 8)
show(p)